### HEROKUのポスグレの接続テスト

In [2]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import datetime
import time
import unicodedata
import string
import requests
import mysql.connector
from PIL import Image, ImageFilter,ImageFont,ImageDraw
from pandas import json_normalize
#import PySimpleGUI as sg
import requests
from time import sleep
import pandas as pd
import unicodedata
import string
import urllib
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#③実行する
import gspread
import json
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
import ssl
from datetime import datetime as dt
ssl._create_default_https_context = ssl._create_unverified_context
# from wordpress_xmlrpc import Client, WordPressPost
# from wordpress_xmlrpc.methods.users import GetUserInfo
# from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
# from utils import *
import datetime
import time
from concurrent.futures import ThreadPoolExecutor

In [3]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)
#自動コミットモードにする

conn.autocommit = True
conn.autocommit
# カーソルを取得する
cursor = conn.cursor()





def get_driver():
    users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
    dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
    passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
    host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
    port = 5432        # DBが稼働しているポート番号(適宜変更)
    # PostgreSQLへ接続
    conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)
    conn.autocommit = True
    conn.autocommit
    # PostgreSQLにデータ登録
    cursor = conn.cursor()
    return cursor
print('get_driver()')

get_driver()


In [30]:
# 必要モジュールのインポート
import os
from dotenv import load_dotenv
# .envファイルの内容を読み込見込む
load_dotenv()
# os.environを用いて環境変数を表示させます
print(os.environ['SHEET_PROJECT_ID'])
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#認証情報設定
# 辞書オブジェクト。認証に必要な情報をHerokuの環境変数から呼び出している
credential = {
                "type": "service_account",
                "project_id": os.environ['SHEET_PROJECT_ID'],
                "private_key_id": os.environ['SHEET_PRIVATE_KEY_ID'],
                "private_key": os.environ['SHEET_PRIVATE_KEY'],
                "client_email": os.environ['SHEET_CLIENT_EMAIL'],
                "client_id": os.environ['SHEET_CLIENT_ID'],
                "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                "token_uri": "https://oauth2.googleapis.com/token",
                "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
                "client_x509_cert_url":  os.environ['SHEET_CLIENT_X509_CERT_URL']
             }
credentials = ServiceAccountCredentials.from_json_keyfile_dict(credential, scope)
gc = gspread.authorize(credentials)
workbook = gc.open_by_key('1x1I73587T9AGO_qhgKO-_qbvO0DC8DXw00ema0oCGJc')



In [6]:
def get_area_sql_text(target_area_name):
    print(target_area_name)
    hokkaidoutouhoku_list = ['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県']
    kitakantou_list = ['茨城県', '栃木県', '群馬県']
    minamikantou_list = ['埼玉県', '千葉県', '東京都', '神奈川県']
    hokurikukoushinetsu_list = ['新潟県', '富山県', '石川県', '福井県', '長野県', '山梨県']
    toukai_list = ['愛知県', '岐阜県', '静岡県', '三重県']
    kansai_list = ['大阪府', '京都府', '兵庫県', '奈良県', '滋賀県', '和歌山県']
    chugokushikoku_list = ['鳥取県', '島根県', '岡山県', '広島県','徳島県', '香川県', '愛媛県', '高知県']
    kyushu_list = [ '山口県','福岡県', '佐賀県', '長崎県', '大分県', '熊本県', '宮崎県', '鹿児島県','沖縄県']
    prefecture_str_lists = ['hokkaidoutouhoku_list', 'kitakantou_list','minamikantou_list','hokurikukoushinetsu_list','toukai_list','kansai_list','chugokushikoku_list','kyushu_list'] 
    for prefecture_list_name in prefecture_str_lists:
        #print(prefecture_list_name)
        if target_area_name == prefecture_list_name.replace('_list',''):
            print('ok',target_area_name)
            target_area_name_list = eval(prefecture_list_name)
            #print(target_area_name_list)
            break
    area_sql_text = ''
    for prefecture_name in target_area_name_list:
        if len(area_sql_text) == 0:
            area_sql_text += f"都道府県 = '{prefecture_name}'"
        else:
            area_sql_text += f" OR 都道府県 = '{prefecture_name}'"
    #print(area_sql_text)
    return area_sql_text

area_sql_text = get_area_sql_text('minamikantou')
print(area_sql_text)

minamikantou
ok minamikantou
都道府県 = '埼玉県' OR 都道府県 = '千葉県' OR 都道府県 = '東京都' OR 都道府県 = '神奈川県'


In [7]:
area_sql_text = get_area_sql_text('minamikantou')
cursor = get_driver()

minamikantou
ok minamikantou


In [8]:
def post_line(message):
    url = "https://notify-api.line.me/api/notify"
    token = os.environ['LINE_TOKEN']
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 
area_sql_text = get_area_sql_text('minamikantou')
cursor = get_driver()
sql = f'''SELECT イベント日,都道府県,店舗名,取材名,取材ランク,媒体名,latitude,longitude
        FROM schedule as schedule2
        left join halldata as halldata2
        on schedule2.店舗名 = halldata2.hall_name
        WHERE イベント日 > current_date
        AND イベント日 <= current_date + 7
        AND 媒体名 != 'ホールナビ'
        AND 媒体名 != '旧イベ'
        AND ({area_sql_text})
        ORDER BY イベント日,都道府県,店舗名,媒体名,取材名 desc;'''#AND (取材ランク = 'S' OR 取材ランク = 'A')
print(sql)
cursor.execute(sql)
result = cursor.fetchall()
report_df = pd.DataFrame(result, columns=['イベント日','都道府県','店舗名','取材名','取材ランク','媒体名','latitude','longitude'])
report_df.to_csv('csv/top_location_df.csv',encoding='utf_8_sig',index=False)
    
#report_df = report_df.dropna(subset=['latitude'])

minamikantou
ok minamikantou
SELECT イベント日,都道府県,店舗名,取材名,取材ランク,媒体名,latitude,longitude
        FROM schedule as schedule2
        left join halldata as halldata2
        on schedule2.店舗名 = halldata2.hall_name
        WHERE イベント日 > current_date
        AND イベント日 <= current_date + 7
        AND 媒体名 != 'ホールナビ'
        AND 媒体名 != '旧イベ'
        AND (都道府県 = '埼玉県' OR 都道府県 = '千葉県' OR 都道府県 = '東京都' OR 都道府県 = '神奈川県')
        ORDER BY イベント日,都道府県,店舗名,媒体名,取材名 desc;


In [25]:
report_df['イベント日']

0      2023-12-01
1      2023-12-01
2      2023-12-01
3      2023-12-01
4      2023-12-01
          ...    
548    2023-12-07
549    2023-12-07
550    2023-12-07
551    2023-12-07
552    2023-12-07
Name: イベント日, Length: 553, dtype: object

In [46]:
type(datetime.date.today())

datetime.date

In [47]:
report_df[:1]["イベント日"].values[0]

numpy.datetime64('2023-12-01T00:00:00.000000000')

In [8]:
import numpy as np
tomorrow = np.datetime64('today', 'D')+ np.timedelta64(1, 'D') 
  
type(tomorrow)

numpy.datetime64

In [9]:
report_df = pd.read_csv('csv/top_location_df.csv', parse_dates=['イベント日'])
if report_df[:1]["イベント日"].values[0] == np.datetime64('today', 'D') :
    print('今日のデータは取得済み')
else:
    print('今日のデータは未取得')
    post_line('今日のデータは未取得です')

from datetime import date, timedelta
today = date.today()
tomorrow = today + timedelta(days=1)
datetime64_tomorrow = np.datetime64(tomorrow)
report_df[report_df['イベント日'] == datetime64_tomorrow ]

FileNotFoundError: [Errno 2] No such file or directory: 'csv/top_location_df.csv'

In [41]:
type(report_df[:1]["イベント日"].values[0])

numpy.datetime64

In [16]:
#文字列に変換
np.datetime64('today', 'D').astype(str)

'2023-12-05'

In [8]:




#area_sql_text = get_area_sql_text('minamikantou')
#首都圏のイベントの媒体別の予約数を集計
#print(area_sql_text)
cursor.execute(f'''SELECT COUNT(店舗名), 店舗名,都道府県
        FROM schedule as schedule2
        left join halldata as halldata2
        on schedule2.店舗名 = halldata2.hall_name
        WHERE イベント日 >= current_date
        AND イベント日 <= current_date + 7
        AND 媒体名 != 'ホールナビ'
        AND 媒体名 != '旧イベ'
        AND ({area_sql_text})
        GROUP BY 店舗名,都道府県
        ORDER BY COUNT(店舗名) desc;''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
#table_df = extract_media_name_df[['イベント日','都道府県','店舗名','取材名']]
#table_df['イベント日'] = table_df['イベント日'].map(convert_sql_date_to_jp_date)
#table_df.drop_duplicates(keep='first',inplace=True)
extract_media_name_df = extract_media_name_df [['count','店舗名','都道府県']]

extract_media_name_df 

,count,店舗名,都道府県
0,14,ガーデン北戸田,埼玉県
1,13,メガガーデン所沢スロット館,埼玉県
2,10,パラッツォ湘南台店,神奈川県
3,10,新！ガーデン四季の森店,神奈川県
4,10,新！ガーデン戸塚,神奈川県
...,...,...,...
254,1,ジャラン谷在家店,東京都
255,1,ジャンジャンデルノザウルス久喜インター店,埼玉県
256,1,ジャンジャンマールゴット南阿佐ヶ谷,東京都
257,1,スーパーDステーション上里,埼玉県


In [61]:
cursor = get_driver()
#area_sql_text = get_area_sql_text('minamikantou')
#首都圏のイベントの媒体別の予約数を集計
#print(area_sql_text)
cursor.execute(f'''SELECT COUNT(店舗名), 店舗名
        FROM schedule as schedule2
        left join halldata as halldata2
        on schedule2.店舗名 = halldata2.hall_name
        WHERE イベント日 >= current_date
        AND イベント日 <= current_date + 7
        AND 媒体名 != 'ホールナビ'
        AND 媒体名 != '旧イベ'
        AND ({area_sql_text})
        GROUP BY 店舗名
        ORDER BY COUNT(店舗名) desc;''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
#table_df = extract_media_name_df[['イベント日','都道府県','店舗名','取材名']]
#table_df['イベント日'] = table_df['イベント日'].map(convert_sql_date_to_jp_date)
#table_df.drop_duplicates(keep='first',inplace=True)
area_sql_text = get_area_sql_text('minamikantou')
extract_media_name_df

minamikantou
ok minamikantou


,count,店舗名
0,39,メガガーデン所沢スロット館
1,28,新！ガーデン西浦和
2,28,ガーデン北戸田
3,21,新！ガーデン戸塚
4,21,パラッツォ湘南台店
...,...,...
361,1,ドラゴン小田急相模原店
362,1,将軍与野店
363,1,SAP相模原
364,1,スーパー夢らんど


In [68]:
int((extract_media_name_df[6:9].index[0]/ 3) + 1)

3

In [8]:
import pandas as pd
import datetime
import time
import unicodedata
import string
import requests
import mysql.connector
import os

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from chromedriver_py import binary_path # this will get you the path variable
from bs4 import BeautifulSoup
import urllib
import urllib.parse
import urllib.request
import re
from sshtunnel import SSHTunnelForwarder
import pymysql as db
import pandas as pd
import datetime
import sshtunnel
import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 



def delete_data(cnx,day):
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()

prefecture_list = ['東京都','静岡県','岐阜県','愛知県','三重県','新潟県','富山県','石川県','福井県','山梨県','長野県','神奈川県','千葉県','埼玉県','群馬県','栃木県','茨城県','福島県','山形県','秋田県','宮城県','岩手県','青森県','北海道']#,'東京都''静岡県','岐阜県','愛知県','三重県'
line_token = os.getenv('LINE_TOKEN')
#print(line_token)
for prefecture in prefecture_list:
    try:
        from selenium.webdriver.chrome.service import Service
        from selenium import webdriver
        from chromedriver_py import binary_path # this will get you the path variable
        svc = Service(executable_path=binary_path)
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        browser = webdriver.Chrome(service=svc, options=options)
        #post_line_text(f'{prefecture}XサーバーへのMYSQL追加処理を開始します',line_token)
        cols = ['店舗名','URL']
        yesterday = datetime.date.today() + datetime.timedelta(days=-1)
        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture}/'
        browser.get(url)
        concat_df = pd.DataFrame(index=[], columns=[])
        parlar_name_parlar_url_dict = {}
        html = browser.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        for element in soup.find(class_="hall-list-table").find_all("a"):
            #print(element.string,element['href'])
            parlar_name = element.string
            parlar_name_url = element['href']
            parlar_name_parlar_url_dict[parlar_name] = parlar_name_url
            record_df = pd.DataFrame([[parlar_name,parlar_name_url]], columns=cols)
            record_df['url_店舗名'] = record_df['URL'].map(lambda x: urllib.parse.unquote((x.split('/')[-2])).replace('-データ一覧',''))
            record_df['都道府県'] = prefecture
            concat_df = pd.concat([concat_df,record_df],axis=0)
            #display(record_df)
        #break
        time.sleep(1)

        #post_line_text(f'{prefecture}は{str(len(concat_df))}の店舗が掲載されいます',line_token)
        count = 0
        error_count = 0
        cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
        ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
        for i, tenpo_name in enumerate(concat_df['url_店舗名']):#tenpo_ichiran_df['ホール名']
            try:
                #time.sleep(1)
                print(i,tenpo_name)
                url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{yesterday.strftime("%Y-%m-%d")}-{tenpo_name}'
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                table = soup.find(id = "all_data_table")
                dfs =pd.read_html(str(table))
                #display(tenpo_df)
                #time.sleep(1)
                for df in  dfs:
                    try:
                        if '機種名' in list(df.columns):
                            ichiran_df = df
                            ichiran_df['日付'] = yesterday.strftime('%Y-%m-%d')
                            ichiran_df['店舗名'] = tenpo_name
                            #print(tenpo_name)
                            ichiran_df['Nのつく日'] = yesterday.strftime('%d')[-1]
                            ichiran_df['都道府県'] = prefecture 
                            ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                            ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, ichiran_df])
                            print('成功',i,tenpo_name)
                            break
                        else:
                            print('見つかりませんでした',i,tenpo_name)
                        count += 1
                    except Exception as e:
                        print(tenpo_name,e)
                        error_count += 1
                        #time.sleep(1)
                        continue
            except Exception as e:
                print(tenpo_name,e)
                continue

            if i > 2:
                break
        cols = ichiran_all_tennpo_df.columns.tolist()
        cols = cols[-4:] + cols[:-4]
        ichiran_all_tennpo_df = ichiran_all_tennpo_df[cols]  #    OR    df = df.ix[:, cols]
        ichiran_all_tennpo_df['ART']= ichiran_all_tennpo_df['ART'].fillna(0)
        ichiran_all_tennpo_df['BB']= ichiran_all_tennpo_df['BB'].fillna(0)
        ichiran_all_tennpo_df['RB']= ichiran_all_tennpo_df['RB'].fillna(0)
        ichiran_all_tennpo_df['差枚']= ichiran_all_tennpo_df['差枚'].fillna(0)
        ichiran_all_tennpo_df['G数']= ichiran_all_tennpo_df['G数'].fillna(0)
        ichiran_all_tennpo_df = ichiran_all_tennpo_df.fillna('')
        #print(ichiran_all_tennpo_df.iloc[:5])
        # SSH 接続 踏み台接続
    except Exception as e:
        print(e)
        #post_line_text(f'{prefecture}のMYSQL追加処理に失敗しました',line_token)
        continue
    break

#     blog.post_line(f'\n{traceback.format_tb(tb)}\n\n{e}')

0 123n東雲店
成功 0 123n東雲店
1 123笹塚店
成功 1 123笹塚店
2 aqua-h2o
成功 2 aqua-h2o
3 arrow池上店
arrow池上店 No tables found
4 bbステーション日暮里店
成功 4 bbステーション日暮里店


In [16]:
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)
def insert_data_bulk(df,conm):
    insert_sql = f"""INSERT INTO public.parlar_data (店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = conn.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    conn.commit()
    print("Insert bulk data")
insert_data_bulk(ichiran_all_tennpo_df,conn)

KeyboardInterrupt: 

In [ ]:
cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']

In [19]:

conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する


def heroku_insert_data_bulk(df,conn):
    cursor = conn.cursor()
    insert_sql = f"""INSERT INTO parlar_data (店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cursor = get_driver()
    cursor.executemany(insert_sql, df.values.tolist())
    print("Insert heroku_bulk data")
    conn.commit()
    cursor.close()
heroku_insert_data_bulk(ichiran_all_tennpo_df[:1],conn)


Insert heroku_bulk data


In [27]:
cursor = get_driver()
#area_sql_text = get_area_sql_text('minamikantou')
#首都圏のイベントの媒体別の予約数を集計
#print(area_sql_text)
cursor.execute(f'''SELECT  *
        FROM parlar_data
        limit 10;''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
#table_df = extract_media_name_df[['イベント日','都道府県','店舗名','取材名']]
#table_df['イベント日'] = table_df['イベント日'].map(convert_sql_date_to_jp_date)
#table_df.drop_duplicates(keep='first',inplace=True)

extract_media_name_df


,店舗名,日付,nのつく日,都道府県,機種名,台番号,g数,差枚,bb,rb,art,bb確率,rb確率,art確率,合成確率,id


In [56]:
cursor.execute('''SELECT *
               FROM halls
               WHERE prefecture = '神奈川県' 
               OR prefecture = '東京都'
                OR prefecture = '埼玉県'
                OR prefecture = '千葉県'
               ORDER BY prefecture''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)

maptable_df


maptable_df['prefecture'] = pd.Categorical(maptable_df['prefecture'], categories=[ '東京都', '神奈川県','埼玉県', '千葉県'], ordered=True)
maptable_df.sort_values('prefecture', inplace=True)
maptable_df


UndefinedTable: relation "halls" does not exist
LINE 2:                FROM halls
                            ^


In [2]:
minami_kantou_list:list[str] = [ '東京都', '神奈川県','埼玉県', '千葉県']



In [35]:
def convert_sql_date_to_jp_date(sql_str_date):
    sql_str_date = str(sql_str_date)
    return sql_str_date.split('-')[1].lstrip('0') + '月' + sql_str_date.split('-')[2].lstrip('0') + '日'

In [28]:
area_name_and_str__jp_area_name_dict = {'hokkaidoutouhoku':'北海道・東北', 'kitakantou':'北関東','minamikantou':'南関東','hokurikukoushinetsu':'北陸・甲信越','toukai':'東海','kansai':'関西','chugokushikoku':'中国・四国','kyushu':'九州・山口'}

data['area_jp_str'] = area_name_and_str__jp_area_name_dict[area_name]
print(area_jp_str)

北海道・東北


In [9]:
cursor = get_driver()
#首都圏のイベントの媒体別の予約数を集計
syuzai_name = 'あつまる応援地区'
cursor.execute(f'''SELECT *
               FROM schedule as schedule2
               left join halldata as halldata2
               on schedule2.店舗名 = halldata2.hall_name
            WHERE イベント日 >= current_date
                AND イベント日 < current_date + 7
                AND 媒体名 != 'ホールナビ'
                AND 取材名 = '{syuzai_name}'
                AND ({area_sql_text})
                ORDER BY イベント日,都道府県 desc;''')
cols = [col.name for col in cursor.description]
extract_syuzai_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
extract_syuzai_name_df

,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間,id,...,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name,bicycle_parking


In [32]:
def create_media_map_iframe(report_df:pd.DataFrame):
    report_df = report_df.drop_duplicates(keep='first')
    report_df = report_df.dropna(subset=['latitude'])
    #(取材ランク = 'S' OR 取材ランク = 'A')のみ抽出
    map_report_df = report_df[['店舗名','取材名','媒体名']].drop_duplicates(keep='first')
    map_report_df = map_report_df.sort_values(['店舗名','媒体名']).reset_index(drop=True)
    #東京都に設定
    prefecture_latitude = 35.68944
    prefecture_longitude = 139.69167
    
    folium_map = folium.Map(location=[prefecture_latitude,prefecture_longitude], zoom_start=10, width="100%", height="100%")
    # 地図表示
    # マーカープロット（ポップアップ設定，色変更，アイコン変更）
    print(report_df)
    for tenpo_name in report_df['店舗名'].unique():
        print(tenpo_name)
        extract_syuzai_df_1 = report_df[report_df['店舗名'] == tenpo_name]
        #display(extract_syuzai_df_1)
        print(extract_syuzai_df_1)
        syuzai_rank_list = list(extract_syuzai_df_1['取材ランク'].unique())
        #print(syuzai_rank_list)
        longitude = extract_syuzai_df_1.iloc[0]['longitude']
        latitude = extract_syuzai_df_1.iloc[0]['latitude']
        print('latitude,longitude',latitude,longitude)
        # グレースケールの画像データを作成
        im= Image.new("L", (280, 100),color=(0))
        im.putalpha(0)
        im2= Image.new("L", (260, 50),color=(50))
        im2.putalpha(128)
        im3 = Image.open('icon.png')
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype('font/LightNovelPOPv2.otf',19)
        if len(extract_syuzai_df_1)==1:
            syuzai_name_text = '◆' + tenpo_name + f'\n {extract_syuzai_df_1["取材名"].values[0]}'
        else:
            syuzai_name_text = '◆' + tenpo_name + f'\n {extract_syuzai_df_1["取材名"].values[0]}、他{len(extract_syuzai_df_1)-1}件'
        #print(syuzai_name_text)
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype('font/LightNovelPOPv2.otf',19)

        # 画像を表示
        im.paste(im3, (-15,-14))
        im.paste(im2, (25,48))
        draw.multiline_text(
            (150, 50),
            f'{syuzai_name_text}',
            font=font,
            fill='white',
            align='center',
            spacing=0,
            anchor='ma'
        )

        im.save('syuzai_image.png', quality=95)
        img = 'syuzai_image.png'
        popup_df = extract_syuzai_df_1[['店舗名','取材名','媒体名']].sort_values('店舗名').reset_index(drop=True).T
        popup_df = popup_df.to_html(escape=False)
        popup_data = folium.Popup(popup_df,  max_width=1500,show=False,size=(700, 300))

        folium.Marker(location=[latitude ,longitude],
            tiles='https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png',
            attr='国土地理院',
            popup=popup_data,
            icon = CustomIcon(
                        icon_image = img,
                        icon_size = (280, 100),
                        icon_anchor = (30, 0),
                        #shadow_image = shadow_img, # 影効果（今回は使用せず コメントアウト
                        #shadow_size = (30, 30),
                        shadow_anchor = (-4, -40),
                        popup_anchor = (3, 3))).add_to(folium_map)
        #break
    
    # set the iframe width and height
    plugins.Fullscreen(
                        position="topright",
                        title="拡大する",
                        title_cancel="元に戻す",
                        force_separate_button=True,
                    ).add_to(folium_map)
    folium_map.get_root().width = "500px"
    folium_map.get_root().height = "600px"
    return folium_map.get_root()._repr_html_()
data['iframe'] = create_media_map_iframe(report_df)

False

In [ ]:
#首都圏のイベントの媒体別の予約数を集計

cursor.execute(f'''SELECT イベント日,都道府県,店舗名,取材名,媒体名,latitude,longitude
               FROM schedule as schedule2
               left join maptable as maptable2
               on schedule2.店舗名 = maptable2.hallnavi_name
               WHERE イベント日 > CURRENT_DATE
                AND イベント日 <= CURRENT_DATE + 1
                AND 媒体名 != 'ホールナビ'
                AND (取材ランク = 'S' OR 取材ランク = 'A')
                AND ({area_sql_text})
                ORDER BY イベント日,都道府県,店舗名,媒体名,取材名 desc;''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
extract_media_name_df['イベント日'] = extract_media_name_df['イベント日'].map(convert_sql_date_to_jp_date)
extract_media_name_df

In [4]:
cursor.execute('''SELECT dmm_name,hallnavi_name, prefecture, latitude
               FROM maptable
               WHERE prefecture = '神奈川県' 
               OR prefecture = '東京都'
                OR prefecture = '埼玉県'
                OR prefecture = '千葉県'
               ORDER BY prefecture''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
maptable_df['table_name'] = 'maptable'
maptable_df

UndefinedTable: relation "maptable" does not exist
LINE 2:                FROM maptable
                            ^


In [57]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE 都道府県 = '神奈川県' 
               OR 都道府県 = '東京都'
                OR 都道府県 = '埼玉県'
                OR 都道府県 = '千葉県'
               ORDER BY 都道府県''')
cols = [col.name for col in cursor.description]
tenpo_master_df = pd.DataFrame(cursor.fetchall(),columns=cols)
tenpo_master_df['table_name'] = 'tenpo_master'
tenpo_master_df.rename(columns={'店舗名／法人名':'dmm_name','都道府県':'prefecture'},inplace=True)
tenpo_master_df

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [66]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度
               FROM tenpo_master
               WHERE 経度 is NULL ''')
cols = [col.name for col in cursor.description]
df = pd.DataFrame(cursor.fetchall(),columns=cols)
df

,店舗名／法人名,都道府県,緯度,経度
0,アンダーツリー株式会社,大阪府,None,None
1,コシード株式会社,,None,None
2,株式会社千歳観光,,None,None
3,BASE株式会社,,None,None
4,ガイドワークス,,None,None
...,...,...,...,...
114,トキワホール,,None,None
115,パーラーデルダス,,None,None
116,Super D’station前橋大利根店,群馬県,None,None
117,yout小牧店,愛知県,None,None


In [222]:
dmm_tenpo_name.replace("'", "''")

"Super D''station錦糸町店"

In [3]:

cols = [col.name for col in cursor.description]
halldata_df = pd.DataFrame(cursor.fetchall(),columns=cols)
halldata_df

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,machine_move,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name
0,10003,アリーナ羽伏,東京都,https://hall-navi.com/hole_view?hid=1000400000...,https://p-town.dmm.com/shops/tokyo/1027,,,,,東京都新島村字南桧山4-1,...,,,,,,,,139.256958,34.377151,None
1,10004,パーラーみやけ坪田店,東京都,https://hall-navi.com/hole_view?hid=1001101000...,https://p-town.dmm.com/shops/tokyo/1028,,,,,東京都三宅島三宅村坪田2987,...,,,,,,,,139.548615,34.061646,None
2,10008,アイランド秋葉原,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/9,http://www.p-world.co.jp/tokyo/island.htm,https://lin.ee/n24S9oy,https://twitter.com/akiba_island,秋葉原,東京都千代田区外神田4-3-3,...,可能,可能,無し,無し,可能,PAPIMO,無し,139.771698,35.700665,None
3,10009,ビッグアップル.秋葉原店,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/7,http://www.p-world.co.jp/tokyo/big-apple.htm,https://line.me/R/ti/p/%40ozs2880u,https://twitter.com/ba_akiba,秋葉原,東京都千代田区外神田1-16-1,...,可能,可能,無し,無し,可能,PAPIMO,無し,139.771362,35.697231,None
4,10013,みとや神田西口店,東京都,https://hall-navi.com/hole_view?hid=1010047000...,https://p-town.dmm.com/shops/tokyo/20,http://www.p-world.co.jp/tokyo/mitoya-kandanis...,,,神田,東京都千代田区内神田2-9-8,...,,,,,,,,139.768311,35.691174,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7234,17212,西の丸船越店,宮崎県,https://hall-navi.com/hole_view?hid=8890603000...,https://p-town.dmm.com/shops/miyazaki/10929,,,,,宮崎県東臼杵郡門川町加草86−1,...,,,,,,,,131.662674,32.499847,None
7235,17215,ナンニチ新栄店,鹿児島県,https://hall-navi.com/hole_view?hid=8900068000...,https://p-town.dmm.com/shops/kagoshima/10984,http://www.p-world.co.jp/kagoshima/nannichi-sh...,,,,鹿児島県鹿児島市東郡元町10−2,...,,,,,,,,130.550476,31.552856,None
7236,17217,ニューマルエイ,鹿児島県,https://hall-navi.com/hole_view?hid=8910515000...,https://p-town.dmm.com/shops/kagoshima/11047,http://www.p-world.co.jp/kagoshima/newmaruei.htm,,,,鹿児島県指宿市山川小川3338,...,,,,,,,,130.601776,31.203983,None
7237,17223,丸栄パチンコ串良店,鹿児島県,https://hall-navi.com/hole_view?hid=8931602000...,https://p-town.dmm.com/shops/kagoshima/11005,http://www.p-world.co.jp/kagoshima/maruei-kush...,,,,鹿児島県鹿屋市串良町有里40-1,...,,,,,,,,130.924286,31.415581,None


In [ ]:
cursor.execute(f'''SELECT *
               FROM halldata as halldata2
               left join maptable as maptable2
               on halldata2.hall_name = maptable2.hallnavi_name
               where maptable2.latitude   is not  null''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
extract_media_name_df

In [ ]:
cursor.execute(f'''SELECT hall_name,COUNT(*) cnt
                FROM   halldata
                GROUP BY hall_name
                HAVING COUNT(*) > 1''')
cols = [col.name for col in cursor.description]
extract_media_name_df = pd.DataFrame(cursor.fetchall(),columns=cols)
extract_media_name_df


In [69]:
cursor.execute(f"""SELECT *
               FROM halldata 
               where (prefecture_name != '神奈川県' 
               and prefecture_name != '東京都'
                and prefecture_name!= '埼玉県'
                and prefecture_name != '千葉県')""")
cols = [col.name for col in cursor.description]
halldata_df = pd.DataFrame(cursor.fetchall(),columns=cols)
halldata_df.reset_index(inplace=True,drop=True)
halldata_df

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,machine_move,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name
0,4786,ノースオメガ,岡山県,https://hall-navi.com/hole_view?hid=7080821000...,,,,,,岡山県津山市野介代39−1,...,,,,,,,,None,None,None
1,2000,PARLOR ACTY&PIAA,大阪府,https://hall-navi.com/hole_view?hid=5300001000...,https://p-town.dmm.com/shops/osaka/7521,,,,,大阪府大阪市北区梅田1丁目2−2,...,,,,,,,,None,None,None
2,4209,パーラーファミリー,富山県,https://hall-navi.com/hole_view?hid=9392306000...,https://p-town.dmm.com/shops/toyama/8935,,,,,富山県富山市八尾町井田4614-3,...,,,,,,,,None,None,None
3,4210,ヤマトII,富山県,https://hall-navi.com/hole_view?hid=9392723000...,https://p-town.dmm.com/shops/toyama/8936,,,,,富山県富山市婦中町萩島313,...,,,,,,,,None,None,None
4,4211,ノースランド2001山室店,富山県,https://hall-navi.com/hole_view?hid=9398005000...,https://p-town.dmm.com/shops/toyama/8937,,,,,富山県富山市中市1丁目6,...,,,,,,,,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676,6573,ジャムフレンドクラブ野辺地,青森県,https://hall-navi.com/hole_view?hid=0393128000...,https://p-town.dmm.com/shops/aomori/1755,http://www.p-world.co.jp/aomori/jamfrendclub-n...,,,,青森県上北郡野辺地町上前田3−1,...,,,,,,,,None,None,None
4677,6574,北大野辺地店,青森県,https://hall-navi.com/hole_view?hid=0393171000...,https://p-town.dmm.com/shops/aomori/1752,http://www.p-world.co.jp/aomori/hokudai-n.htm,,,,青森県上北郡野辺地町二本木13−1,...,,,,,,,,None,None,None
4678,6575,ニュー三光ホール,青森県,https://hall-navi.com/hole_view?hid=0394401000...,https://p-town.dmm.com/shops/aomori/1726,,,,,青森県むつ市大畑町新町2-1,...,,,,,,,,None,None,None
4679,6576,パチンコ大光,青森県,https://hall-navi.com/hole_view?hid=0394601000...,https://p-town.dmm.com/shops/aomori/1756,,,,,青森県下北郡大間町大間上野18-1,...,,,,,,,,None,None,None


In [94]:
new_tenpo_name_list = []
for tenpo_name in list(hallnavi_scraping_error_tenpo_name_dict.keys()):
    new_tenpo_name_list.append(tenpo_name.replace("'", ""))

In [90]:
hallnavi_scraping_error_tenpo_name_list = ["M'sFactory エムズファクトリー",
 "スーパーD'ステーション高崎本店",
 "ハイパーM's",
 "J'S",
 "K'ZONE鳳",
 "K'ZONE",
 "スーパーD'ステーション 新開地店",
 "Aパンクラブ'92",
 'ケイズクラブ',
 "D'station上田店",
 "D's club 河渡店",
 "D's club 笹口店",
 "D's club 燕三条店",
 "D's club 新津店",
 "D's club 分水店",
 "D's club 五泉店",
 "D's club 水原店",
 "Pia'91(ぴあ91)",
 'フェニックス鳥取店',
 "D'グランデ",
 "M's666菊水店",
 "パーラーマンモス'21月寒店",
 "SLOT BAR M's",
 'ダイナム富谷店',
 'パチンコタイガー富谷店',
 'カネマン日高',
 'ダイナム水沢店',
 'WINS奥州',
 'アズ水沢',
 'ウインズ奥州店ライト館',
 'マルハン水沢店',
 "パチンコCAT's北上店"]

In [2]:
import pandas as pd
prefecture_df = pd.read_csv(r'csv/pref_lat_lon.csv')

prefecture_latitude = prefecture_df[prefecture_df['prefecture'] == prefecture]['latitude'].values[0]
prefecture_longitude = prefecture_df[prefecture_df['prefecture'] == prefecture]['longitude'].values[0]

,pref_name,lat,lon
0,北海道,43.064359,141.347449
1,青森県,40.824294,140.740054
2,岩手県,39.703530,141.152667
3,宮城県,38.268737,140.872183
4,秋田県,39.718175,140.103356
5,山形県,38.240127,140.362533
6,福島県,37.750146,140.466754
7,茨城県,36.341817,140.446796
8,栃木県,36.565750,139.883526
9,群馬県,36.391205,139.060917


In [5]:
space_contains_df = halldata_df[halldata_df['hall_name'].str.contains(' ')]
space_contains_df 

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,machine_move,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name
110,10135,スーパーDステーション 錦糸町店,東京都,https://hall-navi.com/hole_view?hid=1300022000...,,http://www.p-world.co.jp/tokyo/dstation-kinshi...,https://line.me/R/ti/p/%40100jxbzj,,錦糸町,東京都墨田区江東橋4-25-5,...,,,,,,,,139.816986,35.695621,None
113,10138,スロット館 プラザ,東京都,https://hall-navi.com/hole_view?hid=1330051000...,https://p-town.dmm.com/shops/tokyo/730,http://www.p-world.co.jp/tokyo/hiranumapuraza.htm,,,京成小岩,東京都江戸川区北小岩6-15-19,...,,,,,,,,139.884705,35.742054,None
133,10160,TOYO HALL,東京都,https://hall-navi.com/hole_view?hid=1350016000...,https://p-town.dmm.com/shops/tokyo/178,http://www.p-world.co.jp/tokyo/toyohall.htm,https://line.me/R/ti/p/%40elf0862k,,木場,東京都江東区東陽3-6-9,...,,,,,,,,139.810669,35.669510,None
153,10181,DELFEEL NEW TOKYO,東京都,https://hall-navi.com/hole_view?hid=1400014000...,https://p-town.dmm.com/shops/tokyo/201,http://www.p-world.co.jp/tokyo/delfeel.htm,https://line.me/R/ti/p/%40xqh2347j,,,東京都品川区大井1-15-2,...,,,,,,,,139.730789,35.607815,None
194,10231,MATSUYA SLOT ZONE,東京都,https://hall-navi.com/hole_view?hid=1530051000...,https://p-town.dmm.com/shops/tokyo/236,http://www.p-world.co.jp/tokyo/matsuya2.htm,,,中目黒,東京都目黒区上目黒3-7-6,...,不明,不明,不明,不明,不明,P'scube,無し,139.698090,35.643776,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7142,16475,Be-1 佐賀店,佐賀県,https://hall-navi.com/hole_view?hid=8400031000...,https://p-town.dmm.com/shops/saga/11378,http://www.p-world.co.jp/saga/be-1saga.htm,,,,佐賀県佐賀市下田町1−30,...,,,,,,,,130.276306,33.246162,None
7165,16735,SLOT ZONE RUNEX 鶴崎店,大分県,https://hall-navi.com/hole_view?hid=8700104000...,https://p-town.dmm.com/shops/oita/10250,http://www.p-world.co.jp/oita/slot-zone-runex.htm,,,,大分県大分市南鶴崎1丁目1−1,...,,,,,,,,131.687424,33.240891,None
7170,16788,パチンコ ダイヤ,大分県,https://hall-navi.com/hole_view?hid=8760844000...,https://p-town.dmm.com/shops/oita/10326,http://www.p-world.co.jp/oita/daiya.htm,,,,大分県佐伯市向島1086-1,...,,,,,,,,131.898254,32.956722,None
7190,16900,西の丸 川南店,宮崎県,https://hall-navi.com/hole_view?hid=8891301000...,https://p-town.dmm.com/shops/miyazaki/10925,,,,,宮崎県児湯郡川南町川南20336,...,,,,,,,,131.486862,32.210018,None


'pwd' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [3]:
#print(tenpo_ichiran_df['ホール名'])
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import urllib.parse
from concurrent.futures import ThreadPoolExecutor
import time
import requests
import pandas as pd
from pandas.api.types import CategoricalDtype
from src.data_loader import DataLoader
from src.blog_utils import Blog
from src.image_utils import ImageUtils
import os
import shutil
import sys
log_file_path = "log/log.txt"
sys.stdout = open(log_file_path, "w", encoding="utf-8") # 表示内容の出力をファイルに変更

prefecture_name_df = pd.read_csv('csv/pref_lat_lon.csv')
decode_df = pd.read_csv('csv/名デコードテーブル都道府県付き.csv')
old_event_df = pd.read_csv('csv/old_event_day.csv')
old_event_df = old_event_df.astype(str)
location_df = pd.read_csv(r'csv\アナスロとホールナビの店舗名比較_マスター.csv')
#old_event_df['旧イベント日'] =old_event_df['旧イベント日'].map(lambda x: x.replace('の付く日',''))
blog = Blog()
tag_json = blog .wp_get_tag_list()
tag_df = pd.DataFrame(tag_json)
tag_df['id'] = tag_df['id'].astype(int)
tag_df = tag_df.sort_values('id')
city_convert_df = pd.read_csv(r'csv\市区郡コード.csv')
# city_convert_df = city_convert_df[city_convert_df['都道府県'] == prefecture_name]
# city_convert_df = city_convert_df[city_convert_df['市区町村'].isnull() == False]
# city_convert_df


dataloader = DataLoader()
image_name_convert_df = dataloader.conver_image_df()

# maptable_df = dataloader.load_heroku_db_to_df(f'''SELECT * FROM maptable''')
# print('maptable_df読み込み完了 maptable_df',maptable_df.shape)
#halldata_df = dataloader.load_heroku_db_to_df(f'''SELECT * FROM halldata''')

import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()




#自動コミットモードにする

conn.autocommit = True
conn.autocommit
def get_driver():
    users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
    dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
    passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
    host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
    port = 5432        # DBが稼働しているポート番号(適宜変更)
    # PostgreSQLへ接続
    conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

    # PostgreSQLにデータ登録
    cursor = conn.cursor()
    return cursor
print('get_driver()')

load_dotenv()
読み込み完了


In [2]:
prefecture_name_df = pd.read_csv('csv/pref_lat_lon.csv')
decode_df = pd.read_csv('csv/名デコードテーブル都道府県付き.csv')
old_event_df = pd.read_csv('csv/old_event_day.csv')
old_event_df = old_event_df.astype(str)
anaslo_hallnavi_convert_df = pd.read_csv(r'csv\アナスロとホールナビの店舗名比較_マスター.csv')
#old_event_df['旧イベント日'] =old_event_df['旧イベント日'].map(lambda x: x.replace('の付く日',''))
blog = Blog()
tag_json = blog .wp_get_tag_list()
tag_df = pd.DataFrame(tag_json)
tag_df['id'] = tag_df['id'].astype(int)
tag_df = tag_df.sort_values('id')
city_convert_df = pd.read_csv(r'csv\市区郡コード.csv')
# city_convert_df = city_convert_df[city_convert_df['都道府県'] == prefecture_name]
# city_convert_df = city_convert_df[city_convert_df['市区町村'].isnull() == False]
# city_convert_df


dataloader = DataLoader()
image_name_convert_df = dataloader.conver_image_df()

# maptable_df = dataloader.load_heroku_db_to_df(f'''SELECT * FROM maptable''')
# print('maptable_df読み込み完了 maptable_df',maptable_df.shape)

halldata_df = dataloader.load_heroku_db_to_df(f'''SELECT * FROM halldata''')
print('halldata_df読み込み完了 halldata_df',halldata_df.shape)

NameError: name 'pd' is not defined

In [48]:
halldata_df = dataloader.load_heroku_db_to_df(f'''SELECT prefecture_name,anaslo_name,address FROM halldata''')
print('halldata_df読み込み完了 halldata_df',halldata_df.shape)
halldata_df['city_name'] = halldata_df['address'].map(extraxt_city_name)
halldata_df = halldata_df[halldata_df['anaslo_name'] != '']
halldata_df
import re

def extraxt_city_name(address):
    pattern = '''(...??[都道府県])((?:旭川|伊達|石狩|盛岡|奥州|田村|南相馬|那須塩原|東村山|武蔵村山|羽村|十日町|上越|
    富山|野々市|大町|蒲郡|四日市|姫路|大和郡山|廿日市|下松|岩国|田川|大村|宮古|富良野|別府|佐伯|黒部|小諸|塩尻|玉野|
    周南)市|(?:余市|高市|[^市]{2,3}?)郡(?:玉村|大町|.{1,5}?)[町村]|(?:.{1,4}市)?[^町]{1,4}?区|.{1,7}?[市町村])(.+)'''
    result = re.match(pattern, address)
    try:
        return result.group(2)
    except:
        return ''

halldata_df.drop_duplicates(subset=['anaslo_name'],keep='first',inplace=True)
halldata_df.drop(columns=['address'],inplace=True)
halldata_df.rename(columns={'prefecture_name':'都道府県','anaslo_name':'店舗名','city_name':'市区町村'},inplace=True)
halldata_df = halldata_df[['都道府県','市区町村','店舗名']]

In [50]:
halldata_df.to_csv('csv/ana_prefecture_hallname_city_name_df.csv',index=False)

['id',
 'hall_name',
 'prefecture_name',
 'hall_url',
 'dmm_url',
 'pworld_url',
 'line_url',
 'twitter_url',
 'nearest_station',
 'address',
 'pachinko_num',
 'slot_num',
 'entrance_method',
 'machine_move',
 'dedama_share',
 'car_parking',
 'bicycle_parking ',
 'mobile_charging',
 'dedama_info',
 'access_info',
 'longitude',
 'latitude',
 'anaslo_name',
 'bicycle_parking']

In [28]:
merge_df

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name,bicycle_parking
0,10003,アリーナ羽伏,東京都,https://hall-navi.com/hole_view?hid=1000400000...,https://p-town.dmm.com/shops/tokyo/1027,,,,,東京都新島村字南桧山4-1,...,,,None,,,,139.256958,34.377151,NaN,
1,10004,パーラーみやけ坪田店,東京都,https://hall-navi.com/hole_view?hid=1001101000...,https://p-town.dmm.com/shops/tokyo/1028,,,,,東京都三宅島三宅村坪田2987,...,,,None,,,,139.548615,34.061646,NaN,
2,10008,アイランド秋葉原,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/9,http://www.p-world.co.jp/tokyo/island.htm,https://lin.ee/n24S9oy,https://twitter.com/akiba_island,秋葉原,東京都千代田区外神田4-3-3,...,可能,無し,None,可能,PAPIMO,無し,139.771698,35.700665,アイランド秋葉原店,無し
3,10009,ビッグアップル.秋葉原店,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/7,http://www.p-world.co.jp/tokyo/big-apple.htm,https://line.me/R/ti/p/%40ozs2880u,https://twitter.com/ba_akiba,秋葉原,東京都千代田区外神田1-16-1,...,可能,無し,None,可能,PAPIMO,無し,139.771362,35.697231,ビッグアップル．秋葉原店,無し
4,10013,みとや神田西口店,東京都,https://hall-navi.com/hole_view?hid=1010047000...,https://p-town.dmm.com/shops/tokyo/20,http://www.p-world.co.jp/tokyo/mitoya-kandanis...,,,神田,東京都千代田区内神田2-9-8,...,,,None,,,,139.768311,35.691174,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,17223,丸栄パチンコ串良店,鹿児島県,https://hall-navi.com/hole_view?hid=8931602000...,https://p-town.dmm.com/shops/kagoshima/11005,http://www.p-world.co.jp/kagoshima/maruei-kush...,,,,鹿児島県鹿屋市串良町有里40-1,...,,,None,,,,130.924286,31.415581,NaN,
7250,17229,イエスランド伊集院店,鹿児島県,https://hall-navi.com/hole_view?hid=8992506000...,https://p-town.dmm.com/shops/kagoshima/11075,http://www.p-world.co.jp/kagoshima/yesland-iju...,,,,鹿児島県日置市伊集院町清藤2051,...,,,None,,,,130.424454,31.627369,NaN,
7251,10450,メッセ昭島店,東京都,https://hall-navi.com/hole_view?hid=1960021000...,https://p-town.dmm.com/shops/tokyo/879,https://www.p-world.co.jp/tokyo/messe-akishima...,https://lin.ee/mNx5h5K,,,東京都昭島市武蔵野3-11-33,...,,,None,,,,139.372665,35.719574,NaN,
7252,15482,ピークスゼロ(柳崎),埼玉県,https://hall-navi.com/hole_view?hid=3330861000...,https://p-town.dmm.com/shops/saitama/3464,http://www.p-world.co.jp/saitama/peaks.htm,https://line.me/R/ti/p/%40uxh6265a,,,埼玉県川口市柳崎5-1-54,...,,,None,,,,139.704773,35.857246,ピークスZERO,


In [31]:
merge_df = pd.merge(halldata_df,anaslo_hallnavi_convert_df,left_on='hall_name',right_on='hallnavi_name',how='left')
merge_df['anaslo_name'] = merge_df['店舗名']
merge_df = merge_df[['id','hall_name','prefecture_name','hall_url','dmm_url','pworld_url','line_url','twitter_url','nearest_station','address','pachinko_num','slot_num','entrance_method','machine_move','dedama_share','car_parking','bicycle_parking ','mobile_charging','dedama_info','access_info','longitude','latitude','anaslo_name','bicycle_parking']]
merge_df.to_csv('csv/halldata_df_20231129.csv',index=False)

In [4]:
halldata_df

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,dedama_share,car_parking,bicycle_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name,bicycle_parking
0,10003,アリーナ羽伏,東京都,https://hall-navi.com/hole_view?hid=1000400000...,https://p-town.dmm.com/shops/tokyo/1027,,,,,東京都新島村字南桧山4-1,...,,,None,,,,139.256958,34.377151,,
1,10004,パーラーみやけ坪田店,東京都,https://hall-navi.com/hole_view?hid=1001101000...,https://p-town.dmm.com/shops/tokyo/1028,,,,,東京都三宅島三宅村坪田2987,...,,,None,,,,139.548615,34.061646,,
2,10008,アイランド秋葉原,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/9,http://www.p-world.co.jp/tokyo/island.htm,https://lin.ee/n24S9oy,https://twitter.com/akiba_island,秋葉原,東京都千代田区外神田4-3-3,...,可能,無し,None,可能,PAPIMO,無し,139.771698,35.700665,,無し
3,10009,ビッグアップル.秋葉原店,東京都,https://hall-navi.com/hole_view?hid=1010021000...,https://p-town.dmm.com/shops/tokyo/7,http://www.p-world.co.jp/tokyo/big-apple.htm,https://line.me/R/ti/p/%40ozs2880u,https://twitter.com/ba_akiba,秋葉原,東京都千代田区外神田1-16-1,...,可能,無し,None,可能,PAPIMO,無し,139.771362,35.697231,,無し
4,10013,みとや神田西口店,東京都,https://hall-navi.com/hole_view?hid=1010047000...,https://p-town.dmm.com/shops/tokyo/20,http://www.p-world.co.jp/tokyo/mitoya-kandanis...,,,神田,東京都千代田区内神田2-9-8,...,,,None,,,,139.768311,35.691174,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7234,17217,ニューマルエイ,鹿児島県,https://hall-navi.com/hole_view?hid=8910515000...,https://p-town.dmm.com/shops/kagoshima/11047,http://www.p-world.co.jp/kagoshima/newmaruei.htm,,,,鹿児島県指宿市山川小川3338,...,,,None,,,,130.601776,31.203983,,
7235,17223,丸栄パチンコ串良店,鹿児島県,https://hall-navi.com/hole_view?hid=8931602000...,https://p-town.dmm.com/shops/kagoshima/11005,http://www.p-world.co.jp/kagoshima/maruei-kush...,,,,鹿児島県鹿屋市串良町有里40-1,...,,,None,,,,130.924286,31.415581,,
7236,17229,イエスランド伊集院店,鹿児島県,https://hall-navi.com/hole_view?hid=8992506000...,https://p-town.dmm.com/shops/kagoshima/11075,http://www.p-world.co.jp/kagoshima/yesland-iju...,,,,鹿児島県日置市伊集院町清藤2051,...,,,None,,,,130.424454,31.627369,,
7237,10450,メッセ昭島店,東京都,https://hall-navi.com/hole_view?hid=1960021000...,https://p-town.dmm.com/shops/tokyo/879,https://www.p-world.co.jp/tokyo/messe-akishima...,https://lin.ee/mNx5h5K,,,東京都昭島市武蔵野3-11-33,...,,,None,,,,139.372665,35.719574,,


In [28]:
halldata_df = pd.read_csv(r'csv\アナスロとホールナビの店舗名比較_マスター.csv')


In [6]:
hallnavi_scraping_error_tenpo_name_dict = {}
import urllib.parse
import requests
import time

for _,row in merge_df.iterrows():
    id_num = row['id']
    tenpo_name = row['hall_name']
    prefecture_name = row['prefecture_name']
    print(id_num,tenpo_name,prefecture_name)
    #print(response.json()[0]["geometry"]["coordinates"])# [経度,緯度]
    # try:
    #     #print(longtitude,latitude)
    #     cursor.execute(f'''UPDATE halldata
    #                 SET hall_name = '{convert_hall_name}'
    #             WHERE  hall_name = '{tenpo_name}'
    #             and prefecture_name = '{prefecture_name}'
    #             ''')
    #     conn.commit()
    #     print('ok',id_num,tenpo_name,convert_hall_name)
    # except Exception as e:
    #     hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
    #     print('エラー店舗',id_num,tenpo_name,e)
    #     #break
    #     continue
    #time.sleep(1)
    break

In [ ]:
extract_hall_name_df.iloc[0].T

slot_num = extract_hall_name_df.iloc[0]['slot_num']
pachinko_num = extract_hall_name_df.iloc[0]['pachinko_num']
adress = extract_hall_name_df.iloc[0]['address']


In [4]:
#-*- using:utf-8 -*-
import urllib.request, urllib.error

def checkURL(url):
    print("開始")
    try:
        f = urllib.request.urlopen(url)
        print ("OK:" + url )
        f.close()
    except:
        print ("NotFound:" + url)

# url='https://www.slo-map.com/tomorrow_recommend/hokkaidoutouhoku/hall/21SEIKI%20%E5%85%AB%E6%88%B8%E8%A5%BF'
# checkURL(url)
url='https://www.slo-map.com/tomorrow_recommend/hokkaidoutouhoku/hall/スーパードーム中央店'
checkURL(url)

開始
NotFound:https://www.slo-map.com/tomorrow_recommend/hokkaidoutouhoku/hall/スーパードーム中央店


In [47]:

groupby_date_kisyubetu_df[['日付','機種名','機種名','総台数','G数','差枚']]
kisyubetu_win_daissuu_list = []
groupby_date_kisyubetu_df_list = []
for kisyu_name,day in zip(groupby_date_kisyubetu_df['機種名'],groupby_date_kisyubetu_df['日付']):
    kisyu_df = extract_hall_status_df.query('機種名 == @kisyu_name & 日付 == @day')
    groupby_date_kisyubetu_df_list.append(kisyu_df)
    kisyu_win_daisuu = len(kisyu_df[kisyu_df['差枚'] > 0])
    kisyubetu_win_daissuu_list.append(kisyu_win_daisuu)
groupby_date_kisyubetu_df['勝率'] = kisyubetu_win_daissuu_list
groupby_date_kisyubetu_df['勝利台数'] = kisyubetu_win_daissuu_list
groupby_date_kisyubetu_df['勝率'] = groupby_date_kisyubetu_df['勝率'].astype(str)
groupby_date_kisyubetu_df['総台数'] = groupby_date_kisyubetu_df['総台数'].astype(int)
groupby_date_kisyubetu_df['平均G数'] = groupby_date_kisyubetu_df['G数'] / groupby_date_kisyubetu_df['総台数']
groupby_date_kisyubetu_df['平均G数'] = groupby_date_kisyubetu_df['平均G数'].astype(int)
groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[groupby_date_kisyubetu_df['総台数'] >= 2 ]
groupby_date_kisyubetu_df['差枚'] = groupby_date_kisyubetu_df['差枚'].astype(int)
groupby_date_kisyubetu_df['平均差枚'] = groupby_date_kisyubetu_df['差枚'] / groupby_date_kisyubetu_df['総台数']
groupby_date_kisyubetu_df['平均差枚'] = groupby_date_kisyubetu_df['平均差枚'].astype(int)
groupby_date_kisyubetu_df['総台数'] = groupby_date_kisyubetu_df['総台数'].astype(str)
groupby_date_kisyubetu_df['勝率'] = groupby_date_kisyubetu_df['勝率'] + '/' + groupby_date_kisyubetu_df['総台数']
groupby_date_kisyubetu_df['勝率'] = groupby_date_kisyubetu_df['勝率'].map(lambda x : '(' + x + '台) ' + str(round(int(x.split('/')[0])/int(x.split('/')[1])*100,1))  + '%')
groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[['日付','機種順位','機種名','勝率','平均G数','平均差枚','差枚','G数','勝利台数','総台数']]
groupby_date_kisyubetu_df = groupby_date_kisyubetu_df.sort_values('平均差枚',ascending=False)
groupby_date_kisyubetu_df['機種平均出率'] =(((groupby_date_kisyubetu_df['G数'] * 3) + groupby_date_kisyubetu_df['差枚']) / (groupby_date_kisyubetu_df['G数'] * 3) )*100
groupby_date_kisyubetu_df['機種平均出率'] = groupby_date_kisyubetu_df['機種平均出率'].map(lambda x : round(x,1))
groupby_date_kisyubetu_df['機種平均出率'] = groupby_date_kisyubetu_df['機種平均出率'].astype(str) + '%'
groupby_date_kisyubetu_df = groupby_date_kisyubetu_df.rename(columns={'G数': '合計G数','差枚': '合計差枚'})
groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[['日付','機種名','勝率','機種平均出率','平均G数','平均差枚','合計差枚','合計G数','勝利台数','総台数']]
# groupby_date_kisyubetu_df['平均G数'] = groupby_date_kisyubetu_df['平均G数'].astype(str) + 'G'
# bubble_chart_samai_division_list = list(groupby_date_kisyubetu_df['合計差枚'][:10])
# groupby_date_kisyubetu_df['平均差枚'] = groupby_date_kisyubetu_df['平均差枚'].astype(str) + '枚'
# groupby_date_kisyubetu_df['合計差枚'] = groupby_date_kisyubetu_df['合計差枚'].astype(str) + '枚'
# groupby_date_kisyubetu_df['合計G数'] = groupby_date_kisyubetu_df['合計G数'].astype(str) + 'G'
# groupby_date_kisyubetu_df['総台数'] = groupby_date_kisyubetu_df['総台数'].astype(str) + '台'
groupby_date_kisyubetu_df.sort_values('日付', ascending=True, inplace=True)
groupby_date_kisyubetu_df.reset_index(drop=True, inplace=True)
groupby_date_kisyubetu_df

,日付,機種名,勝率,機種平均出率,平均G数,平均差枚,合計差枚,合計G数,勝利台数,総台数
0,2023-11-02,劇場版魔法少女まどか☆マギカ[新編]叛逆の物語,(1/2台) 50.0%,98.3%,2251,-117,-234,4503.0,1,2
1,2023-11-02,キングハナハナ-30,(22/42台) 52.4%,101.7%,5596,290,12192,235047.0,22,42
2,2023-11-02,傷物語 ‐始マリノ刻‐,(4/7台) 57.1%,107.1%,1233,261,1830,8636.0,4,7
3,2023-11-02,モンスターハンターワールド:アイスボーン,(2/4台) 50.0%,91.9%,2283,-551,-2207,9133.0,2,4
4,2023-11-02,バーサスリヴァイズ,(0/3台) 0.0%,87.8%,1474,-540,-1622,4422.0,0,3
...,...,...,...,...,...,...,...,...,...,...
131,2023-11-22,炎炎ノ消防隊,(1/4台) 25.0%,89.2%,1598,-519,-2077,6394.0,1,4
132,2023-11-22,ファンキージャグラー2,(4/5台) 80.0%,102.4%,3088,219,1098,15441.0,4,5
133,2023-11-22,沖ドキ!GOLD‐30,(16/32台) 50.0%,104.2%,1729,220,7047,55333.0,16,32
134,2023-11-22,キングハナハナ-30,(9/42台) 21.4%,95.5%,2588,-352,-14787,108727.0,9,42


In [48]:




extract_hall_and_date_machine_df = extract_hall_status_df.groupby(['日付','機種名']).sum().reset_index(drop=False)
extract_hall_and_date_machine_df['台数'] = extract_hall_status_df.groupby(['日付','機種名']).size().reset_index(drop=False)[0]
extract_hall_and_date_machine_df = extract_hall_and_date_machine_df[['日付','機種名','G数','差枚','台数']]
extract_hall_and_date_machine_df['G数'] = extract_hall_and_date_machine_df['G数'].astype(int)
extract_hall_and_date_machine_df['差枚'] = extract_hall_and_date_machine_df['差枚'].astype(int)
extract_hall_and_date_machine_df['平均差枚'] = extract_hall_and_date_machine_df['差枚'] / extract_hall_and_date_machine_df['台数']
extract_hall_and_date_machine_df['平均G数'] = extract_hall_and_date_machine_df['G数'] / extract_hall_and_date_machine_df['台数']
extract_hall_and_date_machine_df['平均差枚'] = extract_hall_and_date_machine_df['平均差枚'].astype(int)
extract_hall_and_date_machine_df['平均G数'] = extract_hall_and_date_machine_df['平均G数'].astype(int)
#groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[['日付','機種名','勝率','平均差枚','平均G数']]
extract_hall_and_date_machine_df['日付'] = pd.to_datetime(extract_hall_and_date_machine_df['日付'])

extract_hall_and_date_machine_df.sort_values('差枚', ascending=False, inplace=True)
#display(extract_hall_and_date_machine_df)
plus_machine_count_df = extract_hall_and_date_machine_df[extract_hall_and_date_machine_df['平均差枚'] > 0].value_counts('機種名').reset_index(drop=False)
#display(plus_machine_count_df)
recommend_machine_list = list(plus_machine_count_df[plus_machine_count_df['count'] == 3]['機種名'])#plus_machine_count_df['count'].max()
#print(recommend_machine_list)
mean_kisyu_df = extract_hall_and_date_machine_df.drop('日付', axis=1)
mean_kisyu_df = mean_kisyu_df.groupby(['機種名']).sum()#.reset_index(drop=False,inplace=True)
mean_kisyu_df['平均差枚'] = mean_kisyu_df['差枚'] / mean_kisyu_df['台数']
mean_kisyu_df.sort_values('平均差枚', ascending=False, inplace=True)
mean_kisyu_df.reset_index(drop=False, inplace=True)
sorted_recommend_kisyu_list = []
for kisyu_name in list(mean_kisyu_df['機種名']):
    if kisyu_name in recommend_machine_list:
        sorted_recommend_kisyu_list.append(kisyu_name)

#print(sorted_recommend_kisyu_list)
n = 1
html_text = ''
for i,kisyu_name in enumerate(sorted_recommend_kisyu_list, start=1):

    extract_machine_name_df= groupby_date_kisyubetu_df[groupby_date_kisyubetu_df['機種名'] == kisyu_name]
    extract_machine_name_df.drop('機種名', axis=1, inplace=True)
    #extract_machine_name_df['日付'] = extract_machine_name_df['日付'].map(self.convert_str_date_to_jp_date_and_weekday)
    #print(kisyu_name)
    average_diff_coin = int((extract_machine_name_df['平均差枚']).mean())
    #table_of_contents_text += f'''                              <li><a href="#{hall_name}_{kisyu_name}">+{average_diff_coin}枚 {kisyu_name}</a></li>\n'''
    html_text += f'<div class="h2 kisyu_001 mt-5 " id="{hall_name}_{kisyu_name}">{kisyu_name}</div>\n'
    #image_url = self.add_convert_string_option_kisyuname(kisyu_name)
    #html_text += image_url
    #画像挿入
    extract_machine_name_df['平均G数'] = extract_machine_name_df['平均G数'].astype(str) + 'G'
    extract_machine_name_df['平均差枚'] = extract_machine_name_df['平均差枚'].astype(str) + '枚'
    #extract_machine_name_df['総台数'] = extract_machine_name_df['総台数'].astype(str) + '台'
    #extract_machine_name_df_1 = extract_machine_name_df[['日付','勝率','平均差枚','平均G数']]
    extract_machine_name_df['勝利台数'] = extract_machine_name_df['勝利台数'].astype(int)
    extract_machine_name_df['総台数'] = extract_machine_name_df['総台数'].astype(int)
    win_number  = (extract_machine_name_df['勝利台数'].sum() / extract_machine_name_df['総台数'].sum()) * 100
    three_times_ave_game = str(int(extract_machine_name_df['合計G数'].sum() / extract_machine_name_df['総台数'].sum())) + 'G'
    three_times_ave_diff_coin = str(int(extract_machine_name_df['合計差枚'].sum() / extract_machine_name_df['総台数'].sum())) + '枚'
    win_number = str(extract_machine_name_df['勝利台数'].sum()) + '/' + str(extract_machine_name_df['総台数'].sum())  + '(' + str(int(round(win_number ,1))) + '%)'
    #print(win_number)
    extract_machine_name_df_1 = extract_machine_name_df[['日付','勝率','平均差枚','平均G数']]
    
    ave_three_times_df = pd.DataFrame([['合計',win_number,three_times_ave_diff_coin,three_times_ave_game]], columns=['日付','勝率','平均差枚','平均G数'])
    concat_machine_name_df = pd.concat([extract_machine_name_df_1,ave_three_times_df],axis=0)
    concat_machine_name_df
    html_text += f'<h1>2023-11-05 {hall_name} お勧め機種{n}位</h1>'
    html_text += concat_machine_name_df.to_html(index=False,escape=False,justify='center') + '\n'
    #display(extract_machine_name_df)
    #word_cloud_machine_name_text_list.append(kisyu_name)
    n += 1

        #break
        

#break


ValueError: cannot convert float NaN to integer

load_dotenv()
読み込み完了


<bound method Series.unique of 110        スーパーDステーション 錦糸町店
113               スロット館 プラザ
133               TOYO HALL
153       DELFEEL NEW TOKYO
194       MATSUYA SLOT ZONE
               ...         
7142               Be-1 佐賀店
7165    SLOT ZONE RUNEX 鶴崎店
7170               パチンコ ダイヤ
7190                西の丸 川南店
7195         テアトロアンジェロ ハレルヤ
Name: hall_name, Length: 870, dtype: object>

In [42]:
len(serched_df)/3

333.0